# Lab 3 - Xavier Cucurull

# NoSQL Databases

In this lab we will focus on MongoDB, a document DB. If you want to read more on the topic you can visit their website [NoSQL explained](https://www.mongodb.com/nosql-explained). 

## Requirements
- Python 3.x
- MongoDB (<https://www.mongodb.com/>)
- pymongo (MongoDB library for Python) (<https://pypi.org/project/pymongo/>)
- Docker Engine

## Setting up the container

Let's pull the Mongo container image from the Docker registry

```
docker pull mongo
```

And start it:
```
docker run --name bda-mongo -e MONGO_INITDB_ROOT_USERNAME=mongoadmin -e MONGO_INITDB_ROOT_PASSWORD=pass1234 -d -p 27017:27017 mongo 
```

Verify the container is running with:
```
docker ps
```

And you should see something like:
```
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                      NAMES
c85eb6cafd15        mongo               "docker-entrypoint.s…"   25 seconds ago      Up 8 seconds        0.0.0.0:27017->27017/tcp   bda-mongo
```

Notice the ports column, Docker is mapping the port 27017 on localhost to the container's port. This means that to reach the DB we just need to connect to localhost:27017.

From now on, you can start and stop the container simply with:
```
docker start bda-mongo
```

```
docker stop bda-mongo
```

## Exercise
We will be working with the same dataset and questions we dealt with in the previous lab, so that you have some reference to work on (and check if your results match).

We will use the [Adult UCI dataset](https://archive.ics.uci.edu/ml/datasets/adult) with a few modifications; download the following file: [data](./files/adults/adults.json)).

Follow the instructions below and answer the questions. 

1. Load the dataset into the DB. How does Mongo treat missing values? Show a few samples.
2. How many people under 18 years old have never worked? Of the never having worked people (all ages) is there any race bias (how many by race)? Is there any sex bias?
3. Look at the hours per week of people with a paying job, by sex. Look at how many's income is above and below 50k. Compare and analyse.
4. How many people with college education do manual labour?
5. What is the minimum, mean and maximum capital gain and capital loss for every marital status?

### 1. Load the dataset into the DB. How does Mongo treat missing values? Show a few samples.

In [1]:
import pymongo
import json
from bson.son import SON

client = pymongo.MongoClient('localhost', 27017, username='mongoadmin', password='pass1234')

# Get already created bda database
database = client['bda']

# let's create a new collection
adults = database.adults

# and load the data
with open('./files/adults/adults.json', 'r') as f:
    data = json.load(f)

In [2]:
print(data['0'])

{'age': 39, 'sex': 'Male', 'race': 'White', 'native_country': 'United_States', 'fnlwgt': 77516, 'education': {'education': 'Bachelors', 'num': 13}, 'relationship': {'marital_status': 'Never_married', 'relationship': 'Not_in_family'}, 'work': {'occupation': 'Adm_clerical', 'hours_per_week': 40, 'workclass': 'State_gov'}, 'finances': {'capital_gain': 2174, 'capital_loss': 0, 'predicted_income': '<=50K'}}


In [3]:
# load the data
adults_list = list(data.values())
result = adults.insert_many(adults_list)

Once the data is loaded into the database let's check that the number of documents corresponds to the number of elements of the input data:

In [4]:
print(len(data))
print(adults.count_documents({}))

32561
32561


Missing values shall be specified as ```null``` in MongoDB. In pymongo these ```null``` values are interpreted as ```None``` so in our case, to find missing values using Python we will query for ```None``` values.

Below we can observe an example wthat the 'native-country' field appears as ```None``` in the data loaded from the JSON.


In [5]:
print(data['14'])

{'age': 40, 'sex': 'Male', 'race': 'Asian_Pac_Islander', 'native_country': None, 'fnlwgt': 121772, 'education': {'education': 'Assoc_voc', 'num': 11}, 'relationship': {'marital_status': 'Married_civ_spouse', 'relationship': 'Husband'}, 'work': {'occupation': 'Craft_repair', 'hours_per_week': 40, 'workclass': 'Private'}, 'finances': {'capital_gain': 0, 'capital_loss': 0, 'predicted_income': '>50K'}, '_id': ObjectId('617ec95891cb9557a47c8273')}


We can query for ```'native-country': None``` and print some of the obtained results.

In [6]:
res = adults.find({'native-country': None}, {'age' : 1, 'native-country': 1})
list(res)[0:5]

[{'_id': ObjectId('617ec95891cb9557a47c8265'), 'age': 39},
 {'_id': ObjectId('617ec95891cb9557a47c8266'), 'age': 50},
 {'_id': ObjectId('617ec95891cb9557a47c8267'), 'age': 38},
 {'_id': ObjectId('617ec95891cb9557a47c8268'), 'age': 53},
 {'_id': ObjectId('617ec95891cb9557a47c8269'), 'age': 28}]

We can do the same for the work occupation.

In [7]:
res = adults.find({'work.occupation': None}, {'work': 1})
list(res)[:5]

[{'_id': ObjectId('617ec95891cb9557a47c8280'),
  'work': {'occupation': None, 'hours_per_week': 60, 'workclass': None}},
 {'_id': ObjectId('617ec95891cb9557a47c82a2'),
  'work': {'occupation': None, 'hours_per_week': 40, 'workclass': None}},
 {'_id': ObjectId('617ec95891cb9557a47c82aa'),
  'work': {'occupation': None, 'hours_per_week': 40, 'workclass': None}},
 {'_id': ObjectId('617ec95891cb9557a47c82b2'),
  'work': {'occupation': None, 'hours_per_week': 2, 'workclass': None}},
 {'_id': ObjectId('617ec95891cb9557a47c82cf'),
  'work': {'occupation': None, 'hours_per_week': 32, 'workclass': None}}]

### 2. How many people under 18 years old have never worked? Of the never having worked people (all ages) is there any race bias (how many by race)? Is there any sex bias?

In [27]:
# People under 18 years old that have never worked
adults.count_documents({'$and': [
                            {'age': {'$lt': 18}},
                            {'work.workclass': 'Never_worked'}
                            ]})

1

There is only one person under 18 years old (17) that has never worked.

In [28]:
# Never-worked grouped by race
pipeline = [
        {'$match': {'work.workclass': 'Never_worked'}},
        {'$group': {'_id': '$race', 'count': {'$sum': 1}}},
        {"$sort": SON([("count", pymongo.DESCENDING)])}
    ]

res = adults.aggregate(pipeline)
for sample in res:
    print(sample)

{'_id': 'White', 'count': 5}
{'_id': 'Black', 'count': 2}


In [29]:
# Compare against whole dataset
pipeline = [
        {'$group': {'_id': '$race', 'count': {'$sum': 1}}},
        {"$sort": SON([("count", pymongo.DESCENDING)])}
    ]

res = adults.aggregate(pipeline)
for sample in res:
    print(sample)

{'_id': 'White', 'count': 27816}
{'_id': 'Black', 'count': 3124}
{'_id': 'Asian_Pac_Islander', 'count': 1039}
{'_id': 'Amer_Indian_Eskimo', 'count': 311}
{'_id': 'Other', 'count': 271}


For the workclass "Never-worked", the ratio of "White" to "Black" is of 2.5, while in the whole dataset "White" population is over-represented almost 9 times more. Although these numbers are not sufficient to confirm a bias it should be taken into account in further analyses.

In [30]:
# Never-worked grouped by sex
pipeline = [
        {'$match': {'work.workclass': 'Never_worked'}},
        {'$group': {'_id': '$sex', 'count': {'$sum': 1}}},
        {"$sort": SON([("count", pymongo.DESCENDING)])}
    ]

res = adults.aggregate(pipeline)
for sample in res:
    print(sample)

{'_id': 'Male', 'count': 5}
{'_id': 'Female', 'count': 2}


In [31]:
# Compare against whole dataset
pipeline = [
        {'$group': {'_id': '$sex', 'count': {'$sum': 1}}},
        {"$sort": SON([("count", pymongo.DESCENDING)])}
    ]

res = adults.aggregate(pipeline)
for sample in res:
    print(sample)

{'_id': 'Male', 'count': 21790}
{'_id': 'Female', 'count': 10771}


We cannot say that there is a sex bias since the proportions are similar, although we can't really confirm or deny it because of the low number of samples.

### 3. Look at the hours per week of people with a paying job, by sex. Look at how many's income is above and below 50k. Compare and analyse.

As in previous labs, let's define *paying job* as all of those jobs which are not of type ```Without_pay``` or ```Never_worked```.

In [32]:
# Average hours-per-week of paying job grouped by sex
pipeline = [
        {'$match': {'$and': [
                            {'work.workclass': {'$ne': 'Without_pay'}},
                            {'work.workclass': {'$ne': 'Never_worked'}},
                            {'work.workclass': {'$ne': None}}
                            ]}},
        {'$group': {'_id': '$sex', 'avgHours': {'$avg': '$work.hours_per_week'}}},
        {"$sort": SON([("avgHours", pymongo.DESCENDING)])}    
]

res = adults.aggregate(pipeline)
for sample in res:
    print(sample)

{'_id': 'Male', 'avgHours': 42.86072477020068}
{'_id': 'Female', 'avgHours': 36.95919395465995}


On average, men work 6 hours per week more than women.

In [33]:
# Average hours-per-week of paying job grouped by sex and workclass
pipeline = [
        {'$match': {'$and': [
                            {'work.workclass': {'$ne': 'Without_pay'}},
                            {'work.workclass': {'$ne': 'Never_worked'}},
                            {'work.workclass': {'$ne': None}}
                            ]}},
        { "$group": {
            "_id": {
                "income": "$finances.predicted_income",
                "sex": "$sex",
            },
            "avgHours": { "$avg": '$work.hours_per_week'}
        }},
    
        {"$sort": SON([("avgHours", pymongo.DESCENDING)])}    
]

res = adults.aggregate(pipeline)
for sample in res:
    print(sample)

{'_id': {'income': '>50K', 'sex': 'Male'}, 'avgHours': 46.550513567376974}
{'_id': {'income': '<=50K', 'sex': 'Male'}, 'avgHours': 41.17241863075196}
{'_id': {'income': '>50K', 'sex': 'Female'}, 'avgHours': 40.817213842058564}
{'_id': {'income': '<=50K', 'sex': 'Female'}, 'avgHours': 36.46499204364628}


We can see a positive correlation between the hours worked and having an income of more than 50K, for both male and female. If we compare both sexes, we see that there is a higher percentage of men that earn more than 50K (31%) compared to women (11%). So it might be the case that the difference in incomes for both sexes is not only related to the hours worked. To further analyse the differences we can look at the hours worked and salaries by workclass.

In [34]:
# Average hours-per-week of paying job grouped by sex and workclass
pipeline = [
        {'$match': {'$and': [
                            {'work.workclass': {'$ne': 'Without_pay'}},
                            {'work.workclass': {'$ne': 'Never_worked'}},
                            {'work.workclass': {'$ne': None}}
                            ]}},
        { "$group": {
            "_id": {
                "income": "$finances.predicted_income",
                "sex": "$sex",
                "workclass": "$work.workclass"
            },
            "avgHours": { "$avg": '$work.hours_per_week'}
        }},
    
        {"$sort": SON([("avgHours", pymongo.DESCENDING)])}    
]

res = adults.aggregate(pipeline)
for sample in res:
    print(sample)

{'_id': {'income': '>50K', 'sex': 'Male', 'workclass': 'Self_emp_inc'}, 'avgHours': 50.82788296041308}
{'_id': {'income': '<=50K', 'sex': 'Male', 'workclass': 'Self_emp_inc'}, 'avgHours': 47.8675}
{'_id': {'income': '>50K', 'sex': 'Male', 'workclass': 'Self_emp_not_inc'}, 'avgHours': 47.85202492211838}
{'_id': {'income': '>50K', 'sex': 'Male', 'workclass': 'Private'}, 'avgHours': 46.314845409487845}
{'_id': {'income': '>50K', 'sex': 'Male', 'workclass': 'State_gov'}, 'avgHours': 45.12280701754386}
{'_id': {'income': '<=50K', 'sex': 'Male', 'workclass': 'Self_emp_not_inc'}, 'avgHours': 45.041333333333334}
{'_id': {'income': '>50K', 'sex': 'Male', 'workclass': 'Local_gov'}, 'avgHours': 44.58099352051836}
{'_id': {'income': '<=50K', 'sex': 'Female', 'workclass': 'Self_emp_inc'}, 'avgHours': 44.02127659574468}
{'_id': {'income': '>50K', 'sex': 'Female', 'workclass': 'Federal_gov'}, 'avgHours': 43.732142857142854}
{'_id': {'income': '>50K', 'sex': 'Male', 'workclass': 'Federal_gov'}, 'avgHo

For example, federal-gov shows 56 women vs. 315 men, making more than 50K, who work the same hours on average. However, for the other workclasses, male that make more than 50K also work more hours on average that female who make more than 50K. Nevertheless, the proportion of men making more than 50K is higher than women for most of the workclasses.

### 4. How many people with college education do manual labour?

First, we define manual labour as those jobs that are not *desk* jobs, being *desk* jobs ```Tech_support```, ```Sales```, ```Exec_managerial```, ```Prof_specialty```, ```Adm_clerical```. Then, to filter people with college education we can make use of the ```education.num``` variable.

In [35]:
# People with college education (or higher) that do manual jobs (don't do desk jobs)
n = adults.count_documents({'$and': [
                            {'education.num': {'$gte': 10}},
                            {'work.occupation': {'$ne': 'Tech_support'}},
                            {'work.occupation': {'$ne': 'Sales'}},
                            {'work.occupation': {'$ne': 'Exec_managerial'}},
                            {'work.occupation': {'$ne': 'Prof_specialty'}},
                            {'work.occupation': {'$ne': 'Adm_clerical'}},
                            {'work.occupation': {'$ne': None}}
                            ]})

# Grouped by occupation
pipeline = [
        {'$match': {'$and': [
                            {'education.num': {'$gte': 10}},
                            {'work.occupation': {'$ne': 'Tech_support'}},
                            {'work.occupation': {'$ne': 'Sales'}},
                            {'work.occupation': {'$ne': 'Exec_managerial'}},
                            {'work.occupation': {'$ne': 'Prof_specialty'}},
                            {'work.occupation': {'$ne': 'Adm_clerical'}},
                            {'work.occupation': {'$ne': None}}
                            ]}},
        { "$group": {
            "_id": {"occupation": "$work.occupation"},
            "count": { "$sum": 1 }
        }},
    
        {"$sort": SON([("count", pymongo.DESCENDING)])}    
]


res = adults.aggregate(pipeline)

for sample in res:
    print(sample)
    
print(f'\n{n} adults with college education do manual labour\n')


{'_id': {'occupation': 'Craft_repair'}, 'count': 1492}
{'_id': {'occupation': 'Other_service'}, 'count': 1179}
{'_id': {'occupation': 'Machine_op_inspct'}, 'count': 485}
{'_id': {'occupation': 'Transport_moving'}, 'count': 426}
{'_id': {'occupation': 'Protective_serv'}, 'count': 400}
{'_id': {'occupation': 'Handlers_cleaners'}, 'count': 374}
{'_id': {'occupation': 'Farming_fishing'}, 'count': 332}
{'_id': {'occupation': 'Priv_house_serv'}, 'count': 30}
{'_id': {'occupation': 'Armed_Forces'}, 'count': 4}

4722 adults with college education do manual labour



### 5. What is the minimum, mean and maximum capital gain and capital loss for every marital status?

In [36]:
# MIN, AVG and MAX capital-gain and capital-loss by marital status
pipeline = [
        { "$group": {
            "_id": {"maritalStatus": "$relationship.marital_status"},
            "min_cap_gain": { "$min": '$finances.capital_gain'},
            "max_cap_gain": { "$max": '$finances.capital_gain'},
            "avg_cap_gain": { "$avg": '$finances.capital_gain'},
            "min_cap_loss": { "$min": '$finances.capital_loss'},
            "max_cap_loss": { "$max": '$finances.capital_loss'},
            "avg_cap_loss": { "$avg": '$finances.capital_loss'},
            'count': {'$sum': 1}
        }},
    
        {"$sort": SON([("count", pymongo.DESCENDING)])}  
]

res = adults.aggregate(pipeline)
for sample in res:
    print(sample)

{'_id': {'maritalStatus': 'Married_civ_spouse'}, 'min_cap_gain': 0, 'max_cap_gain': 99999, 'avg_cap_gain': 1764.8595085470085, 'min_cap_loss': 0, 'max_cap_loss': 2603, 'avg_cap_loss': 122.43910256410257, 'count': 14976}
{'_id': {'maritalStatus': 'Never_married'}, 'min_cap_gain': 0, 'max_cap_gain': 99999, 'avg_cap_gain': 376.58831788823363, 'min_cap_loss': 0, 'max_cap_loss': 3683, 'avg_cap_loss': 52.34550219975662, 'count': 10683}
{'_id': {'maritalStatus': 'Divorced'}, 'min_cap_gain': 0, 'max_cap_gain': 99999, 'avg_cap_gain': 728.4148098131893, 'min_cap_loss': 0, 'max_cap_loss': 3900, 'avg_cap_loss': 64.60072023407608, 'count': 4443}
{'_id': {'maritalStatus': 'Separated'}, 'min_cap_gain': 0, 'max_cap_gain': 99999, 'avg_cap_gain': 535.5687804878049, 'min_cap_loss': 0, 'max_cap_loss': 3900, 'avg_cap_loss': 58.110243902439024, 'count': 1025}
{'_id': {'maritalStatus': 'Widowed'}, 'min_cap_gain': 0, 'max_cap_gain': 99999, 'avg_cap_gain': 571.0715005035247, 'min_cap_loss': 0, 'max_cap_loss': 

To visualize it better we can create a Pandas DataFrame with the obtained command cursor.

In [37]:
import pandas as pd

res = adults.aggregate(pipeline)
df =  pd.DataFrame(list(res))
df

,_id,min_cap_gain,max_cap_gain,avg_cap_gain,min_cap_loss,max_cap_loss,avg_cap_loss,count
0,{'maritalStatus': 'Married_civ_spouse'},0,99999,1764.859509,0,2603,122.439103,14976
1,{'maritalStatus': 'Never_married'},0,99999,376.588318,0,3683,52.345502,10683
2,{'maritalStatus': 'Divorced'},0,99999,728.414810,0,3900,64.600720,4443
3,{'maritalStatus': 'Separated'},0,99999,535.568780,0,3900,58.110244,1025
4,{'maritalStatus': 'Widowed'},0,99999,571.071501,0,4356,79.666667,993
5,{'maritalStatus': 'Married_spouse_absent'},0,99999,653.983254,0,2472,57.779904,418
6,{'maritalStatus': 'Married_AF_spouse'},0,7298,432.652174,0,0,0.000000,23


In order to draw conclusions from this data it is important to consider the number of examples from each category of marital-status. Some of them such as Married-AF-spouse or Married-spouse-absent contain too little examples to be used to extract meaningful knowledge. However, most people are either married with a civil spouse or never married, and from these two groups we clearly see that the average capital-loss and capital-gain are higher for married adults.

Besides that, is a bit strange that the minimum capital gain and loss is always 0. Although not specified in the dataset description, this could in fact indicate missing values for these numeric columns.

Finally, we must close the connection to the database.

In [38]:
client.close()